-   [Transmission on networks](#chap:c11)
    -   [S preamble: objects, classes and functions](#sec:s3)
    -   [Networks](#networks)
    -   [Models of networks](#models-of-networks)
        -   [Watts-Strogatz Networks](#watts-strogatz-networks)
        -   [Barabasi-Albert Networks](#barabasi-albert-networks)
    -   [Epidemics on networks](#epidemics-on-networks)

Transmission on networks
========================

S preamble: objects, classes and functions
------------------------------------------

The S-language which is the foundation of R was constructed using an ‘object’-based logic where each object is assigned a ‘class’. The class, in turn, controls printing, plotting and summarizing each object. There are many excellent introductions to S programming , in this chapter we will use S3-class programming to streamline our analysis of epidemics on networks. The basic idea is this: if we label the result of some calculation as class `foo`, then R will look for functions `print.foo()`, `summary.foo()` and `plot.foo()` in the search-path when further interacting with the result of the calculation. Let’s illustarte with a silly example:

In [ ]:
foo = function(x) {
    res = x
    class(res) = "foo"
    return(res)
}

print.foo = function(obj) {
    cat("foo is:\n", obj)
}

summary.foo = function(obj) {
    cat("In summary, foo is:\n", obj)
}

plot.foo = function(obj) {
    plot(NA, type = "n", ylim = c(0, 1), xlim = c(0, 1), ylab = "")
    text(x = seq(0.1, 0.9, by = 0.1), y = seq(0.1, 0.9, by = 0.1), 
        as.character(obj))
}

The result is a fully functional S3-class of R objects:

In [ ]:
zz = foo("pibble")

which we can print,

In [ ]:
zz

summarize,

In [ ]:
summary(zz)

and plot (fig. \[fig:zz\]):

In [ ]:
plot(zz)

And that is the basics of S3-class programming ...

Networks
--------

Transmission on social networks bears conceptual similarities to spatial transmission. The only difference being that in spatial models transmission occurs among neighbors in space, and transmission on networks occurs among neighbors in social space. We can thus use the type of compact code we used for CML models (section \[sec:c8cml\]) to simulate epidemics on networks. A key determinant of invasibility and speed of spread is the average and variance in the number of contacts on the networks . As we saw in the network of spread of gonorrhea (section \[sec:contra\]), there is often substantial variation in the number of sexual partners. Section \[sec:waifw\] further highlighted age-specific variation in contact rates. It is easiest to consider static networks (networks for which contact patterns do not change over time) for which the contact distribution is characterized by the ‘degree distribution’, contacts are mapped onto ‘edges’ and individuals are ‘nodes’.

Models of networks
------------------

In the previous spatial coupled map lattice models, transmission was restricted to the eight nearest neighbors on the lattice, so we can think of this as an example of a network with fixed degree of 8. In network theory, an analogous fixed-degree network is constructed as a ring lattice. The associated matrix that flags neighbors is a particular type of [](https://en.wikipedia.org/wiki/Toeplitz_matrix). We can define a `ringlattice`-function to generate such networks with `N` nodes and `2*K` degrees. We label the result to be of class `cm` (short for contact matrix):

In [ ]:
ringlattice = function(N, K) {
    # N is the number of nodes K is the number of neighbors on
    # each side to which each node is connected so degree = 2xK
    CM = toeplitz(c(0, rep(1, K), rep(0, N - 2 * K - 1), rep(1, 
        K)))
    class(CM) = "cm"
    return(CM)
}

Trigonometry provides a basic way to visualize a ring network... or any other object that is defined as class `cm`.

In [ ]:
plot.cm = function(CM){
   N = dim(CM)[1]
   theta = seq(0,2*pi,length = N+1)
   x = cos(theta[1:N])
   y = sin(theta[1:N])
   symbols(x,y, fg = 0, circles = rep(1, N), 
      inches = 0.1, bg = 1, xlab = "", ylab = "")
   segx1 = as.vector(matrix(x, ncol = length(x),
       nrow = length(x), byrow = TRUE))
   segx2 = as.vector(matrix(x, ncol = length(x), 
       nrow = length(x), byrow = FALSE))
   segy1 = as.vector(matrix(y, ncol = length(x), 
       nrow = length(x), byrow = TRUE))
   segy2 = as.vector(matrix(y, ncol = length(x), 
       nrow = length(x), byrow = FALSE))
   segments(segx1,segy1, segx2, 
      segy2, lty = as.vector(CM))
}

Figure \[fig:cm1\] depicts a ring-lattice with 20 individuals and a fixed-degree of eight.

In [ ]:
cm = ringlattice(N = 20, K = 4)
plot(cm)

### Watts-Strogatz Networks

Real social networks have heterogeneities in contact rates and usually exhibits much lower social separation than predicted by the ring lattice. In the study of small-world networks, [](https://en.wikipedia.org/wiki/Watts_and_Strogatz_model) proposed an algorithm for generating more realistic networks by randomly rewiring a fraction `Prw` of the edges of a ring lattice.

In [ ]:
WattsStrogatz = function(N, K, Prw){
  # Build a Watts-Strogatz contact matrix from 
  # a ring lattice, Prw is the rewiring probability
  CM = ringlattice(N = N, K = K)
  CMWS = CM
  tri = CM[upper.tri(CM)]
  Br = rbinom(length(tri),1,Prw) # Break edges 
  a = 0
  for(i in 1:(N-1)){                                    
    for(j in (i+1):N){
         a = a+1                                
         if(Br[a]==1 & CMWS[i,j]==1){ #If "Br == 1"
             CMWS[i,j] = CMWS[j,i] = 0 # break edge
             tmp = i
             tmp2 = c(i, which(CMWS[i,]==1))
             #new edge, if already present try again
             while(any(tmp2==tmp)){
                tmp = ceiling(N*runif(1))} 
             CMWS[i,tmp] = CMWS[tmp,i] = 1 # make new edge
             }
    }
  }
class(CMWS) = "cm"
return(CMWS)
}

Figure \[fig:cm2\] depicts a Watts-Strogatz network with 20 individuals, a mean degree of 8 and a rewiring probability of $0.3$.

In [ ]:
cm2 = WattsStrogatz(N = 20, K = 4, Prw = 0.3)
plot(cm2)

We can extend the notion of writing generic functions for class `cm`-objects, to define a `summary`-function that calculates and optionally plots (fig. \[fig:cm3\]) the degree distribution.

In [ ]:
summary.cm = function(x, plot = FALSE) {
    x = table(apply(x, 2, sum))
    res = data.frame(n = x)
    names(res) = c("degree", "freq")
    if (plot) 
        barplot(x, xlab = "degree")
    return(res)
}
summary(cm2, plot = TRUE)

The Watts-Strogatz model scales the degree distribution from fixed to the ‘random graph’ – when the rewiring probability is set to 1 – which has a Poisson-distributed degree distribution. The random graph corresponds to the [](https://en.wikipedia.org/wiki/Erdos-Renyi_model) .

### Barabasi-Albert Networks

The Watts-Strogatz model can at most have Poisson-like variance in degree distribution, so it cannot mimic heavy-tailed distributions seen in ‘scale-free’ networks. [](https://en.wikipedia.org/wiki/Barabasi-Albert_model) () proposed that such behavior arise from preferential attachment (‘rich-get-richer’) dynamics. We can write a function that generates a network with N-nodes and mean degree `2*K`. The log-log plot (fig. \[fig:BA\]) shows the power-law heterogeneity in contacts predicted by the Barabasi-Albert algorithm.

In [ ]:
BarabasiAlbert = function(N, K){
   CM = matrix(0, ncol = N, nrow = N)
   CM[1,2] = 1
   CM[2,1] = 1
   for(i in 3:N){   
       probs = apply(CM, 1, sum)
       link = unique(sample(c(1:N)[-i], 
          size = min(c(K, i-1)), prob = probs[-i]))
      CM[i, link] = CM[link, i] = 1
   }
class(CM) = "cm"
return(CM)
}

cm3 = BarabasiAlbert(200, 4)
ed = summary(cm3)
plot(as.numeric(ed$degree), ed$freq, log = "xy", xlab = "Degree", 
    ylab = "Frequency")

For fancier visualization of networks we can use the plotting functions in the `statnet`-package (fig. \[fig:snet\]). The `network`-function in the `statnet`-package convert the contact matrix (of class `CM`) to a `network`-class object.

In [ ]:
require(statnet)
plot(network(cm3, directed = FALSE))

Epidemics on networks
---------------------

We can run SIR-like epidemics across networks by assuming that an infection is transmitted across an S-I edge with a probability, $\tau$, per time step . Following, the [](https://en.wikipedia.org/wiki/Reed-Frost_model) version of the chain-binomial model , the probability of any given susceptible becoming infected is $p=1-(1-\tau)^y$ where $y$ is the number of infected neighbors. We may further assume infecteds are removed with a constant probabillity, $\gamma$, leading to a geometrically distributed infectious period. Spread of infections on networks depends on both mean number of contacts $\overline{k}$ and heterogeneity in that number (quantified by $\overline{k^2}$) according to $R_0 = (\tau /(\tau+ \gamma)) (\overline{k^2} - \overline{k})/ \overline{k})$ .

The `NetworkSIR` function will simulate a closed SIR epidemic on arbitrary contact matrices and return an object of class `netSIR`.

In [ ]:
NetworkSIR = function(CM,tau,gamma){
  #generate SIR epidemic on a CM-network  
  #CM  =  contact matrix
  #tau  =  probability of infection across an edge
  #gamma  =  probability of removal per time step 
  N = dim(CM)[1]
  I = matrix(rep(0,N),nrow = N,ncol = 1) #First infecteds 
  S = matrix(rep(1,N),nrow = N,ncol = 1) #First susceptibles 
  R = matrix(rep(0,N),nrow = N,ncol = 1) #First removed 
  I1 = sample(1:N, size = 1)#Pick first random infected
  I[I1,1] = 1
  S[I1,1] = 0   
  t = 1
  while(sum(I[,t-1])>0 | t==1){
    t = t+1
    infneigh = CM%*%I[,t-1]
    pinf = 1-(1-tau)^infneigh
    newI = rbinom(N, S[,t-1], pinf)
    newR = rbinom(N, I[,t-1], gamma)
    nextS = S[,t-1]-newI
    nextI = I[,t-1]+newI-newR
    nextR = R[,t-1]+newR
    I = cbind(I, nextI)
    S = cbind(S, nextS)
    R = cbind(R, nextR)
  } 
res = list(I = I,S = S,R = R)
class(res) = "netSIR"
return(res)
}

We can define `summary`- and `plot`-functions for the `netSIR` class.

In [ ]:
summary.netSIR = function(x){
   t = dim(x$S)[2]
   S = apply(x$S,2,sum)
   I = apply(x$I,2,sum)
   R = apply(x$R,2,sum)
   res = data.frame(S = S,I = I,R = R)
   return(res)
}

plot.netSIR = function(x){
   y = summary(x)   
   plot(y$S, type = "b", xlab = "time", ylab = "")
   lines(y$I, type = "b", col = "red")  
   lines(y$R, type = "b", col = "blue") 
   legend("right", legend = c("S", "I", "R"),
     lty = c(1,1,1), pch = c(1,1,1),
     col = c("black", "red", "blue"))
}

Figure \[fig:netsim\] show epidemic spread on (i) scale-free, (ii) Watts-Strogatz, (iii) Poisson, and (iv) ring lattice networks.

In [ ]:
cm1 = BarabasiAlbert(N = 200,K = 2)  #(i)
cm2 = WattsStrogatz(N = 200, K = 2, Prw = .1) #(ii)
cm3 = WattsStrogatz(N = 200, K = 2, Prw = 1)  #(iii)
cm4 = ringlattice(N = 200,K = 2)   #(iv)
sim1 = NetworkSIR(cm1,.3,0.1)
sim2 = NetworkSIR(cm2,.3,0.1)
sim3 = NetworkSIR(cm3,.3,0.1)
sim4 = NetworkSIR(cm4,.3,0.1)
plot(apply(sim1$I,2,sum), type = "l", xlab = "Time", 
     ylab = "Infected")
lines(apply(sim2$I,2,sum), type = "l", col = "red")
lines(apply(sim3$I,2,sum), type = "l", col = "red", lty = 2)
lines(apply(sim4$I,2,sum), type = "l", col = "blue")
legend("topright", legend = c("Scale-free", "WS(0.1)", 
   "Poisson", "Lattice"), lty = c(1,1, 2, 1), 
   col=c("black", "red", "red", "blue"))

The difference in spread can be understood in terms of how network geometry moulds $R_0$ even when all else (including the mean number of contacts) is constant . The `r0fun`- function calculates $R_0$ for any given network and apply it to each simulated networks. The greater the heterogeneity the greater the $R_0$:

In [ ]:
r0fun = function(CM, tau, gamma){
x = apply(CM, 2, sum)
(tau/(tau+gamma))*(mean(x^2)-(mean(x)))/mean(x)
}
r0fun(cm1, 0.3, 0.1)
r0fun(cm2, 0.3, 0.1)
r0fun(cm3, 0.3, 0.1)
r0fun(cm4, 0.3, 0.1)

We can combine the functionality of the `statnet`-package with the above results on network heterogeneity to revisit on the gonorrhea contact tracing study of from section \[sec:contra\].

In [ ]:
data(gonnet)
nwt = network(gonnet, directed = TRUE)
x = degree(nwt)[2:89]
mean(x)

The mean degree is 1.92, but the inflation factor due to the network heterogeneity is predicted to almost double the $R_0$ of a STD spreading across this network:

In [ ]:
(mean(x^2) - (mean(x)))/mean(x)

To simulate an epidemic on the empirical contact-tracing study from section \[sec:contra\], we first have to construct a undirected contact network among the 89 members and next apply the `NetworkSIR` model to plot the time trajectory and final infection status of the network:

In [ ]:
#Undirected network
cmg = gonnet+t(gonnet)
#Simulate epidemid
cep = NetworkSIR(cmg, 0.3, 0.1)
sm = summary(cep)
par(mfrow = c(1,2))
inf = ifelse(apply(cep$I,1,sum)>0,2,1)
nwt = network(cmg, directed = FALSE)
plot(nwt, vertex.col = inf)
matplot(sm, ylab = "Numbers")
legend("right", c("S", "I", "R"), 
     pch = c("1", "2", "3"), col = c(1,2,3))

The infection history of the network (fig. \[fig:netcmg\]) reveals the feature that the core-group of the network is likely to always be infected but peripheral individuals may escape infection by getting surrounded by immune individuals before getting infected. discusses how the geometry of a network shapes the likelihood of a given individual escaping infection.

Models of networks and epidemics on networks is a vast literature, so the above should at best be consider a teaser. The statnet project and associated `statnet`-package has a rich set of resources for network analysis and modeling epidemics on networks including how to generate dynamic networks.